In [1]:
%run base.ipynb

<h1>System Inertia Matrix example</h1>

In [2]:
inertia_mat = uv_body.body_inertia_matrix()
M_func = Function('M_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m], [inertia_mat]) # for numerical & symbolic use

# M_func.save('M.casadi')

In [3]:
inertia_mat

SX(@1=0, 
[[(m-X_du), @1, @1, @1, ((m*z_g)-X_dq), @1], 
 [@1, (m-Y_dv), @1, (-((m*z_g)+Y_dp)), @1, @1], 
 [@1, @1, (m-Z_dw), @1, @1, @1], 
 [@1, (-((m*z_g)+K_dv)), @1, (I_x-K_dp), @1, @1], 
 [((m*z_g)-M_du), @1, @1, @1, (I_y-M_dq), @1], 
 [@1, @1, @1, @1, @1, (I_z-N_dr)]])

In [4]:
# example usage of inertia mat
M_mat = np.array(M_func(blue.m, blue.Io, blue.rg[2], blue.added_m, blue.coupl_added_m))
M_mat

array([[17.  ,  0.  ,  0.  ,  0.  ,  0.23,  0.  ],
       [ 0.  , 24.2 ,  0.  , -0.23,  0.  ,  0.  ],
       [ 0.  ,  0.  , 26.07,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.28,  0.  ,  0.  ],
       [ 0.23,  0.  ,  0.  ,  0.  ,  0.28,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.28]])

In [5]:
M_mat.T == M_mat

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

<h1>Coriolis Matrix example</h1>

In [6]:
coriolis_mat = uv_body.body_coriolis_centripetal_matrix()
C_func = Function('C_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m, x_nb, v_c], [coriolis_mat]) # for numerical & symbolic use

In [7]:
v_r_0 = np.array([0.1 ,0.2,0.1,0.1,0.1,0.5]) # example velocity vector
C = np.array(C_func(blue.m, blue.Io, blue.rg[2], blue.added_m, blue.coupl_added_m, v_r_0, 
                    np.array([0.0 ,0.0,0.0,0.0,0.0,0.0])))
C

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  2.607, -4.817],
       [ 0.   ,  0.   ,  0.   , -2.607,  0.   ,  1.723],
       [ 0.   ,  0.   ,  0.   ,  4.817, -1.723,  0.   ],
       [ 0.   ,  2.607, -4.817,  0.   ,  0.14 , -0.051],
       [-2.607,  0.   ,  1.723, -0.14 ,  0.   , -0.018],
       [ 4.817, -1.723,  0.   ,  0.051,  0.018,  0.   ]])

In [8]:
-C.T == C

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

<h1>Restoring Forces</h1>

In [9]:
res_G = uv_body.body_restoring_vector()
res_G_func = Function('G_n', [z, eul, W, B, r_g, r_b], [res_G]) # for numerical & symbolic use

In [10]:
blue_eul = np.array([0.1, 0.2, 0.3]) # example euler angles
res_G_func(blue.below_surface, blue_eul, blue.W, blue.B, blue.rg, blue.rb)

DM([-0.394359, 0.194219, 1.93571, 0.220764, 0.448258, -0])

In [11]:
# # c , cpp or matlab code generation for forward dynamics
# res_G_func.generate("res_G_func.c")
# os.system(f"gcc -fPIC -shared res_G_func.c -o libg.so")

In [12]:
damping = uv_body.body_damping_matrix()
damping_func = Function('G_n', [x_nb, v_c, linear_dc, quadratic_dc], [damping]) # for numerical & symbolic use

In [13]:
damping_func(v_r_0, blue.v_flow, blue.linear_dc, blue.quadratic_dc)

DM(
[[5.848, 00, 00, 00, 00, 00], 
 [00, 10.552, 00, 00, 00, 00], 
 [00, 00, 8.879, 00, 00, 00], 
 [00, 00, 00, 0.225, 00, 00], 
 [00, 00, 00, 00, 0.225, 00], 
 [00, 00, 00, 00, 00, 0.845]])

In [14]:
bias_force = uv_body.get_bias()
bias_force_func = Function('Bias_b', [z, eul, x_nb ,sim_p, f_ext], [bias_force])
# bias_force_func.save('C.casadi')

# # c , cpp or matlab code generation for dynamics instance
# bias_force_func.generate("C_uv_vec.c")
# os.system(f"gcc -fPIC -shared C_uv_vec.c -o libBias_vehicle.so")

In [15]:
blue_f_ext = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # example external forces
bias_force_func(blue.below_surface, blue_eul, v_r_0, blue.sim_params, blue_f_ext)

DM([-1.95736, 2.90542, 3.13301, 0.271464, 0.359358, 0.5665])

In [16]:
M_mat_func = Function('M_b', [sim_p], [inertia_mat]) # for numerical & symbolic use
# M_mat_func.save('M.casadi')

# # c , cpp or matlab code generation for dynamics instance
# M_mat_func.generate("M_uv_matrix.c")
# os.system(f"gcc -fPIC -shared M_uv_matrix.c -o libMass_vehicle.so")